In [1]:
"""
VNA S-Parameter Report Generation System

This module provides classes for generating comprehensive VNA measurement reports
including S-parameters, TDR/TDT analysis, diagrams, and metadata.

Author: VNA Report Generator
Date: 2026-01-25
"""

import skrf as rf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas as pdf_canvas
from reportlab.lib.units import inch
from reportlab.platypus import Table, TableStyle, Paragraph
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib import colors
from io import BytesIO
from PIL import Image
from attrs import define, field
from datetime import datetime
from typing import Optional


@define
class VNAMetadata:
    """
    VNA measurement configuration and metadata
    
    Attributes
    ----------
    instrument_id : str
        VNA instrument identification string
    if_bandwidth : float
        IF filter bandwidth in Hz
    num_averages : int
        Number of averages per point
    num_points : int
        Number of frequency points in sweep
    start_freq : float
        Start frequency in Hz
    stop_freq : float
        Stop frequency in Hz
    power_level : float
        Source power level in dBm
    measurement_date : str
        Date and time of measurement
    calibration_type : str
        Type of calibration used (e.g., "SOLT", "TRL")
    calibration_date : str, optional
        Date of last calibration
    operator : str, optional
        Name of operator who performed measurement
    notes : str, optional
        Additional notes or comments
    """
    
    instrument_id: str = field(default="Unknown VNA")
    if_bandwidth: float = field(default=1000.0)  # Hz
    num_averages: int = field(default=1)
    num_points: int = field(default=201)
    start_freq: float = field(default=1e9)  # Hz
    stop_freq: float = field(default=10e9)  # Hz
    power_level: float = field(default=0.0)  # dBm
    measurement_date: str = field(factory=lambda: datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    calibration_type: str = field(default="SOLT")
    calibration_date: Optional[str] = field(default=None)
    operator: Optional[str] = field(default=None)
    notes: Optional[str] = field(default=None)
    
    @property
    def span_freq(self) -> float:
        """Frequency span in Hz"""
        return self.stop_freq - self.start_freq
    
    @property
    def step_freq(self) -> float:
        """Frequency step size in Hz"""
        return self.span_freq / (self.num_points - 1) if self.num_points > 1 else 0
    
    @property
    def start_freq_ghz(self) -> float:
        """Start frequency in GHz"""
        return self.start_freq / 1e9
    
    @property
    def stop_freq_ghz(self) -> float:
        """Stop frequency in GHz"""
        return self.stop_freq / 1e9
    
    @property
    def span_freq_ghz(self) -> float:
        """Frequency span in GHz"""
        return self.span_freq / 1e9
    
    @property
    def if_bandwidth_khz(self) -> float:
        """IF bandwidth in kHz"""
        return self.if_bandwidth / 1e3
    
    def __repr__(self) -> str:
        """
        Pretty representation with dotted leaders
        
        Returns
        -------
        str
            Formatted string representation of VNA metadata
        """
        # Define field width for alignment
        field_width = 30
        
        lines = [
            "VNA MEASUREMENT CONFIGURATION",
            "=" * 60,
            "",
        ]
        
        # Helper function to create dotted line
        def format_line(label: str, value: str) -> str:
            dots_needed = field_width - len(label)
            return f"{label}{'.' * dots_needed} {value}"
        
        lines.extend([
            format_line("Instrument", self.instrument_id),
            format_line("Start Frequency", f"{self.start_freq_ghz:.3f} GHz"),
            format_line("Stop Frequency", f"{self.stop_freq_ghz:.3f} GHz"),
            format_line("Frequency Span", f"{self.span_freq_ghz:.3f} GHz"),
            format_line("Frequency Step", f"{self.step_freq/1e6:.3f} MHz"),
            format_line("Number of Points", str(self.num_points)),
            format_line("IF Bandwidth", f"{self.if_bandwidth_khz:.1f} kHz"),
            format_line("Number of Averages", str(self.num_averages)),
            format_line("Power Level", f"{self.power_level:.1f} dBm"),
            format_line("Calibration Type", self.calibration_type),
            format_line("Calibration Date", self.calibration_date or 'N/A'),
            format_line("Measurement Date", self.measurement_date),
            format_line("Operator", self.operator or 'N/A'),
        ])
        
        if self.notes:
            lines.extend([
                "",
                "Notes:",
                "-" * 60,
                self.notes
            ])
        
        return "\n".join(lines)
    
    def to_dataframe(self) -> pd.DataFrame:
        """
        Convert metadata to pandas DataFrame for table display
        
        Returns
        -------
        pd.DataFrame
            Two-column DataFrame with parameter names and values
        """
        data = {
            'Parameter': [
                'Instrument',
                'Start Frequency',
                'Stop Frequency',
                'Frequency Span',
                'Frequency Step',
                'Number of Points',
                'IF Bandwidth',
                'Number of Averages',
                'Power Level',
                'Calibration Type',
                'Calibration Date',
                'Measurement Date',
                'Operator',
            ],
            'Value': [
                self.instrument_id,
                f'{self.start_freq_ghz:.3f} GHz',
                f'{self.stop_freq_ghz:.3f} GHz',
                f'{self.span_freq_ghz:.3f} GHz',
                f'{self.step_freq/1e6:.3f} MHz',
                str(self.num_points),
                f'{self.if_bandwidth_khz:.1f} kHz',
                str(self.num_averages),
                f'{self.power_level:.1f} dBm',
                self.calibration_type,
                self.calibration_date or 'N/A',
                self.measurement_date,
                self.operator or 'N/A',
            ]
        }
        
        df = pd.DataFrame(data)
        return df


class VNAMetadataPage:
    """Page layout for VNA metadata display"""
    
    def __init__(self, metadata: VNAMetadata, use_repr_format: bool = False):
        """
        Parameters
        ----------
        metadata : VNAMetadata
            VNA metadata object to display
        use_repr_format : bool
            If True, use dotted-leader repr format instead of table
        """
        if metadata is None:
            raise ValueError("metadata cannot be None")
        
        self.metadata = metadata
        self.use_repr_format = use_repr_format
        
    def add_to_canvas(self, canvas, header_link=None):
        """
        Add this page to a ReportLab canvas
        
        Parameters
        ----------
        canvas : reportlab.pdfgen.canvas.Canvas
            The PDF canvas to draw on
        header_link : str, optional
            URL or bookmark name for header hyperlink
        """
        page_width, page_height = letter
        
        # Add header with optional hyperlink
        if header_link:
            canvas.linkURL(header_link, (0.5*inch, page_height - 0.4*inch, 
                                         page_width - 0.5*inch, page_height - 0.2*inch))
        
        canvas.setFont("Helvetica-Bold", 10)
        canvas.drawString(0.5*inch, page_height - 0.3*inch, "VNA Configuration")
        
        # Title
        canvas.setFont("Helvetica-Bold", 16)
        canvas.drawCentredString(page_width/2, page_height - 1.5*inch, 
                                "VNA Measurement Configuration")
        
        # Render metadata in chosen format
        if self.use_repr_format:
            self._add_metadata_text(canvas, page_height - 2.2*inch)
        else:
            self._add_metadata_table(canvas, page_height - 2.2*inch)
            # Add notes section if present (only for table format)
            if self.metadata.notes:
                self._add_notes_section(canvas, page_height - 7.5*inch)
    
    def _add_metadata_text(self, canvas, y_position):
        """
        Add metadata as text using repr format with dotted leaders
        
        Parameters
        ----------
        canvas : reportlab.pdfgen.canvas.Canvas
            The PDF canvas
        y_position : float
            Y position for top of text
        """
        page_width = letter[0]
        
        # Get repr text
        repr_text = repr(self.metadata)
        lines = repr_text.split('\n')
        
        # Use monospace font for alignment
        canvas.setFont("Courier", 10)
        
        # Draw each line
        current_y = y_position
        for line in lines:
            # Center the text block
            if line.startswith("VNA MEASUREMENT CONFIGURATION"):
                canvas.setFont("Courier-Bold", 11)
                canvas.drawCentredString(page_width/2, current_y, line)
                canvas.setFont("Courier", 10)
            elif line.startswith("=") or line.startswith("-"):
                canvas.drawCentredString(page_width/2, current_y, line)
            elif line.strip() == "":
                pass  # Just move down
            elif line.startswith("Notes:"):
                canvas.setFont("Courier-Bold", 10)
                canvas.drawString(1.5*inch, current_y, line)
                canvas.setFont("Courier", 10)
            else:
                canvas.drawString(1.5*inch, current_y, line)
            
            current_y -= 0.2*inch
    
    def _add_metadata_table(self, canvas, y_position):
        """
        Add metadata table to canvas
        
        Parameters
        ----------
        canvas : reportlab.pdfgen.canvas.Canvas
            The PDF canvas
        y_position : float
            Y position for top of table
        """
        page_width, page_height = letter
        
        # Convert metadata to DataFrame
        df = self.metadata.to_dataframe()
        
        # Convert DataFrame to list of lists for ReportLab Table
        data = [df.columns.tolist()] + df.values.tolist()
        
        # Create table with wider columns
        table = Table(data, colWidths=[2.5*inch, 4*inch])
        
        # Style the table
        style_commands = [
            # Header row
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#4A90E2')),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (0, -1), 'LEFT'),  # Parameter column left-aligned
            ('ALIGN', (1, 0), (1, -1), 'LEFT'),  # Value column left-aligned
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 12),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('TOPPADDING', (0, 0), (-1, 0), 12),
            
            # Data rows
            ('BACKGROUND', (0, 1), (-1, -1), colors.white),
            ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 1), (-1, -1), 10),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('TOPPADDING', (0, 1), (-1, -1), 8),
            ('BOTTOMPADDING', (0, 1), (-1, -1), 8),
            ('LEFTPADDING', (0, 0), (-1, -1), 10),
            ('RIGHTPADDING', (0, 0), (-1, -1), 10),
        ]
        
        # Alternate row colors for readability
        for i in range(1, len(data)):
            if i % 2 == 0:
                style_commands.append(
                    ('BACKGROUND', (0, i), (-1, i), colors.HexColor('#F0F0F0'))
                )
        
        table.setStyle(TableStyle(style_commands))
        
        # Calculate table position (centered horizontally)
        table_width, table_height = table.wrap(0, 0)
        x_position = (page_width - table_width) / 2
        
        # Draw table
        table.drawOn(canvas, x_position, y_position - table_height)
    
    def _add_notes_section(self, canvas, y_position):
        """
        Add notes section to canvas
        
        Parameters
        ----------
        canvas : reportlab.pdfgen.canvas.Canvas
            The PDF canvas
        y_position : float
            Y position for notes section
        """
        page_width = letter[0]
        
        canvas.setFont("Helvetica-Bold", 12)
        canvas.drawString(1.5*inch, y_position, "Notes:")
        
        canvas.setFont("Helvetica", 10)
        # Wrap text if needed
        text_object = canvas.beginText(1.5*inch, y_position - 0.3*inch)
        text_object.setFont("Helvetica", 10)
        
        # Simple text wrapping
        max_width = page_width - 3*inch
        words = self.metadata.notes.split()
        line = ""
        for word in words:
            test_line = line + word + " "
            if canvas.stringWidth(test_line, "Helvetica", 10) < max_width:
                line = test_line
            else:
                text_object.textLine(line)
                line = word + " "
        if line:
            text_object.textLine(line)
        
        canvas.drawText(text_object)


class DifferentialPairDiagram:
    """Creates diagrams for differential pair S-parameter measurements"""
    
    def __init__(self):
        pass
    
    def generate_twoport_diagram(self, title="2-Port S-Parameters"):
        """
        Generate diagram showing 2-port S-parameters
        
        Returns
        -------
        BytesIO
            Buffer containing PNG image data
        """
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.set_xlim(0, 10)
        ax.set_ylim(0, 6)
        ax.axis('off')
        
        # Title
        ax.text(5, 5.5, title, fontsize=14, fontweight='bold', ha='center')
        
        # VNA Ports
        port1_y = 3
        port2_y = 3
        ax.text(0.5, port1_y, 'VNA Port 1', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue', edgecolor='black'))
        ax.text(9.5, port2_y, 'VNA Port 2', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', edgecolor='black'))
        
        # DUT Box
        dut_box = FancyBboxPatch((3, 2), 4, 2, boxstyle="round,pad=0.1",
                                  edgecolor='black', facecolor='lightyellow', linewidth=2)
        ax.add_patch(dut_box)
        
        # DUT label
        ax.text(5, 3, 'Device Under Test\n(2-Port Network)', fontsize=11, fontweight='bold',
                ha='center', va='center')
        
        # Signal trace
        ax.plot([1.2, 3], [port1_y, 3], 'b-', linewidth=3)
        ax.plot([7, 8.8], [3, port2_y], 'b-', linewidth=3)
        
        # Port 1 (Tx) side
        ax.text(2.5, 4.5, 'Tx', fontsize=11, fontweight='bold', color='darkblue')
        
        # Port 2 (Rx) side  
        ax.text(7.5, 4.5, 'Rx', fontsize=11, fontweight='bold', color='darkgreen')
        
        # S-parameter labels with boxes
        # S11 - Reflection at Port 1
        ax.text(1.8, 3.7, 'S11', fontsize=10, fontweight='bold', color='red',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='red'))
        
        # S22 - Reflection at Port 2
        ax.text(8.2, 3.7, 'S22', fontsize=10, fontweight='bold', color='red',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='red'))
        
        # S21 - Forward transmission (Port 1 to Port 2)
        arrow_fwd = FancyArrowPatch((3.2, 3.4), (6.8, 3.4),
                                    arrowstyle='->', mutation_scale=25, linewidth=2.5,
                                    color='green')
        ax.add_patch(arrow_fwd)
        ax.text(5, 3.8, 'S21', fontsize=10, fontweight='bold', color='darkgreen',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='green'))
        
        # S12 - Reverse transmission (Port 2 to Port 1)
        arrow_rev = FancyArrowPatch((6.8, 2.6), (3.2, 2.6),
                                    arrowstyle='->', mutation_scale=25, linewidth=2.5,
                                    color='orange')
        ax.add_patch(arrow_rev)
        ax.text(5, 2.2, 'S12', fontsize=10, fontweight='bold', color='darkorange',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='orange'))
        
        # Legend/Description
        ax.text(5, 1.2, 'S11, S22: Input/Output Return Loss (Reflection)', 
                fontsize=9, ha='center', style='italic', color='darkred')
        ax.text(5, 0.8, 'S21: Forward Gain/Loss  |  S12: Reverse Isolation', 
                fontsize=9, ha='center', style='italic', color='darkgreen')
        ax.text(5, 0.4, 'For passive reciprocal devices: S12 = S21', 
                fontsize=9, ha='center', style='italic', color='gray')
        
        plt.tight_layout()
        
        # Save to buffer
        buf = BytesIO()
        plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
        buf.seek(0)
        plt.close(fig)
        
        return buf
    
    def generate_single_ended_diagram(self, title="Single-Ended S-Parameters"):
        """
        Generate diagram showing single-ended S-parameters for differential pair
        
        Returns
        -------
        BytesIO
            Buffer containing PNG image data
        """
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.set_xlim(0, 10)
        ax.set_ylim(0, 6)
        ax.axis('off')
        
        # Title
        ax.text(5, 5.5, title, fontsize=14, fontweight='bold', ha='center')
        
        # VNA Ports - Left side (Tx)
        port1_y = 4
        port3_y = 2
        ax.text(0.5, port1_y, 'VNA Port 1', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue', edgecolor='black'))
        ax.text(0.5, port3_y, 'VNA Port 3', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue', edgecolor='black'))
        
        # VNA Ports - Right side (Rx)
        port2_y = 4
        port4_y = 2
        ax.text(9.5, port2_y, 'VNA Port 2', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', edgecolor='black'))
        ax.text(9.5, port4_y, 'VNA Port 4', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', edgecolor='black'))
        
        # Differential Pair Box
        pair_box = FancyBboxPatch((2, 1.5), 6, 3, boxstyle="round,pad=0.1",
                                  edgecolor='black', facecolor='lightyellow', linewidth=2)
        ax.add_patch(pair_box)
        
        # Lane label
        ax.text(5, 3, 'Differential Pair\n(Lane 1)', fontsize=11, fontweight='bold',
                ha='center', va='center')
        
        # Positive trace
        ax.plot([1.2, 2], [port1_y, 4], 'k-', linewidth=2)
        ax.plot([2, 8], [4, 4], 'b-', linewidth=3, label='+ Signal')
        ax.plot([8, 8.8], [4, port2_y], 'k-', linewidth=2)
        
        # Negative trace
        ax.plot([1.2, 2], [port3_y, 2], 'k-', linewidth=2)
        ax.plot([2, 8], [2, 2], 'r-', linewidth=3, label='- Signal')
        ax.plot([8, 8.8], [2, port4_y], 'k-', linewidth=2)
        
        # S-parameter labels
        # Port 1 connections
        ax.text(1.6, 4.3, 'S11', fontsize=9, style='italic', color='blue')
        ax.text(1.6, 3.6, 'S13, S31', fontsize=9, style='italic', color='purple')
        
        # Port 3 connections
        ax.text(1.6, 2.3, 'S33', fontsize=9, style='italic', color='blue')
        
        # Port 2 connections
        ax.text(8.4, 4.3, 'S22', fontsize=9, style='italic', color='blue')
        ax.text(8.4, 3.6, 'S24, S42', fontsize=9, style='italic', color='purple')
        
        # Port 4 connections
        ax.text(8.4, 2.3, 'S44', fontsize=9, style='italic', color='blue')
        
        # Through connections
        ax.text(5, 4.3, 'S21, S12', fontsize=9, style='italic', color='green')
        ax.text(5, 1.7, 'S34, S43', fontsize=9, style='italic', color='green')
        
        # Tx/Rx labels
        ax.text(2.5, 4.7, 'Tx', fontsize=10, fontweight='bold', color='darkblue')
        ax.text(7.5, 4.7, 'Rx', fontsize=10, fontweight='bold', color='darkgreen')
        
        plt.tight_layout()
        
        # Save to buffer
        buf = BytesIO()
        plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
        buf.seek(0)
        plt.close(fig)
        
        return buf
    
    def generate_crosstalk_diagram(self, title="Cross-Coupling S-Parameters"):
        """
        Generate diagram showing cross-coupling terms
        
        Returns
        -------
        BytesIO
            Buffer containing PNG image data
        """
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.set_xlim(0, 10)
        ax.set_ylim(0, 6)
        ax.axis('off')
        
        # Title
        ax.text(5, 5.5, title, fontsize=14, fontweight='bold', ha='center')
        
        # VNA Ports - Left side (Tx)
        port1_y = 4
        port3_y = 2
        ax.text(0.5, port1_y, 'VNA Port 1', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue', edgecolor='black'))
        ax.text(0.5, port3_y, 'VNA Port 3', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue', edgecolor='black'))
        
        # VNA Ports - Right side (Rx)
        port2_y = 4
        port4_y = 2
        ax.text(9.5, port2_y, 'VNA Port 2', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', edgecolor='black'))
        ax.text(9.5, port4_y, 'VNA Port 4', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', edgecolor='black'))
        
        # Differential Pair Box
        pair_box = FancyBboxPatch((2, 1.5), 6, 3, boxstyle="round,pad=0.1",
                                  edgecolor='black', facecolor='lightyellow', linewidth=2)
        ax.add_patch(pair_box)
        
        # Lane label
        ax.text(5, 3, 'Differential Pair\n(Lane 1)', fontsize=11, fontweight='bold',
                ha='center', va='center')
        
        # Positive trace
        ax.plot([1.2, 2], [port1_y, 4], 'k-', linewidth=2)
        ax.plot([2, 8], [4, 4], 'b-', linewidth=3, label='+ Signal')
        ax.plot([8, 8.8], [4, port2_y], 'k-', linewidth=2)
        
        # Negative trace
        ax.plot([1.2, 2], [port3_y, 2], 'k-', linewidth=2)
        ax.plot([2, 8], [2, 2], 'r-', linewidth=3, label='- Signal')
        ax.plot([8, 8.8], [2, port4_y], 'k-', linewidth=2)
        
        # Cross-coupling arrows (dashed)
        # Port 1 to Port 4
        arrow1 = FancyArrowPatch((1.2, port1_y-0.3), (8.8, port4_y+0.3),
                                arrowstyle='->', mutation_scale=20, linewidth=2,
                                color='red', linestyle='--', alpha=0.6)
        ax.add_patch(arrow1)
        ax.text(5, 2.7, 'S14, S41', fontsize=10, style='italic', color='red',
                ha='center', bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
        
        # Port 3 to Port 2
        arrow2 = FancyArrowPatch((1.2, port3_y+0.3), (8.8, port2_y-0.3),
                                arrowstyle='->', mutation_scale=20, linewidth=2,
                                color='orange', linestyle='--', alpha=0.6)
        ax.add_patch(arrow2)
        ax.text(5, 3.3, 'S23, S32', fontsize=10, style='italic', color='orange',
                ha='center', bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
        
        # Description
        ax.text(5, 0.8, 'Cross-coupling between positive and negative lines',
                fontsize=9, ha='center', style='italic', color='darkred')
        
        plt.tight_layout()
        
        # Save to buffer
        buf = BytesIO()
        plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
        buf.seek(0)
        plt.close(fig)
        
        return buf
    
    def generate_mixed_mode_diagram(self, title="Mixed-Mode S-Parameters"):
        """
        Generate diagram showing differential mode S-parameters
        
        Returns
        -------
        BytesIO
            Buffer containing PNG image data
        """
        fig, ax = plt.subplots(figsize=(10, 7))
        ax.set_xlim(0, 10)
        ax.set_ylim(0, 7)
        ax.axis('off')
        
        # Title
        ax.text(5, 6.5, title, fontsize=14, fontweight='bold', ha='center')
        
        # Left side - Differential Mode Ports
        ax.text(0.8, 5, 'Diff Port 1', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightcoral', edgecolor='black'))
        ax.text(0.8, 3, 'Common Port 1', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgray', edgecolor='black'))
        
        # Right side - Differential Mode Ports
        ax.text(9.2, 5, 'Diff Port 2', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightcoral', edgecolor='black'))
        ax.text(9.2, 3, 'Common Port 2', fontsize=10, ha='center', va='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgray', edgecolor='black'))
        
        # Differential Pair Box
        pair_box = FancyBboxPatch((2.5, 2), 5, 3.5, boxstyle="round,pad=0.1",
                                  edgecolor='black', facecolor='lightyellow', linewidth=2)
        ax.add_patch(pair_box)
        
        # Lane label
        ax.text(5, 3.75, 'Differential Pair', fontsize=11, fontweight='bold',
                ha='center', va='center')
        
        # Differential mode traces
        ax.plot([1.5, 2.5], [5, 4.8], 'b-', linewidth=3)
        ax.plot([2.5, 7.5], [4.8, 4.8], 'b-', linewidth=3)
        ax.plot([7.5, 8.5], [4.8, 5], 'b-', linewidth=3)
        
        ax.plot([1.5, 2.5], [5, 2.7], 'r-', linewidth=3)
        ax.plot([2.5, 7.5], [2.7, 2.7], 'r-', linewidth=3)
        ax.plot([7.5, 8.5], [2.7, 5], 'r-', linewidth=3)
        
        # Common mode (dashed)
        ax.plot([1.5, 7.5], [3, 3], 'gray', linewidth=2, linestyle='--', alpha=0.6)
        ax.plot([7.5, 8.5], [3, 3], 'gray', linewidth=2, linestyle='--', alpha=0.6)
        
        # S-parameter labels in boxes
        # Differential
        ax.text(3.5, 5.5, 'Sdd11', fontsize=10, fontweight='bold', color='darkblue',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='blue'))
        ax.text(6.5, 5.5, 'Sdd22', fontsize=10, fontweight='bold', color='darkblue',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='blue'))
        ax.text(5, 5.2, 'Sdd21, Sdd12', fontsize=10, fontweight='bold', color='darkgreen',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='green'))
        
        # Common mode
        ax.text(3.5, 2.3, 'Scc11', fontsize=10, fontweight='bold', color='gray',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='gray'))
        ax.text(6.5, 2.3, 'Scc22', fontsize=10, fontweight='bold', color='gray',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='gray'))
        ax.text(5, 2.5, 'Scc21, Scc12', fontsize=10, fontweight='bold', color='darkgray',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='gray'))
        
        # Mode conversion
        ax.text(2, 3.75, 'Scd11', fontsize=9, style='italic', color='purple')
        ax.text(8, 3.75, 'Scd22', fontsize=9, style='italic', color='purple')
        ax.text(2, 4.2, 'Sdc11', fontsize=9, style='italic', color='purple')
        ax.text(8, 4.2, 'Sdc22', fontsize=9, style='italic', color='purple')
        
        # Legend
        ax.text(5, 1.3, 'dd: Differential → Differential', fontsize=9, ha='center', color='darkblue')
        ax.text(5, 0.9, 'cc: Common → Common', fontsize=9, ha='center', color='gray')
        ax.text(5, 0.5, 'cd/dc: Mode Conversion', fontsize=9, ha='center', color='purple')
        
        plt.tight_layout()
        
        # Save to buffer
        buf = BytesIO()
        plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
        buf.seek(0)
        plt.close(fig)
        
        return buf


class DiagramPage:
    """Page layout for diagrams"""
    
    def __init__(self, figure_buffer, page_title):
        """
        Parameters
        ----------
        figure_buffer : BytesIO
            Pre-generated figure buffer
        page_title : str
            Title for the page
        """
        if figure_buffer is None:
            raise ValueError("figure_buffer cannot be None - generate diagram first")
        
        self.figure_buffer = figure_buffer
        self.page_title = page_title
        
    def add_to_canvas(self, canvas, header_link=None):
        """
        Add this page to a ReportLab canvas
        
        Parameters
        ----------
        canvas : reportlab.pdfgen.canvas.Canvas
            The PDF canvas to draw on
        header_link : str, optional
            URL or bookmark name for header hyperlink
        """
        page_width, page_height = letter
        
        # Add header with optional hyperlink
        if header_link:
            canvas.linkURL(header_link, (0.5*inch, page_height - 0.4*inch, 
                                         page_width - 0.5*inch, page_height - 0.2*inch))
        
        canvas.setFont("Helvetica-Bold", 10)
        canvas.drawString(0.5*inch, page_height - 0.3*inch, self.page_title)
        
        # Load and draw image
        with Image.open(self.figure_buffer) as img:
            img_width, img_height = img.size
            
            margin = 0.5 * inch
            max_width = page_width - 2*margin
            max_height = page_height - 2*margin - 0.5*inch
            
            scale = min(max_width / img_width, max_height / img_height)
            scaled_width = img_width * scale
            scaled_height = img_height * scale
            
            x = (page_width - scaled_width) / 2
            y = (page_height - scaled_height) / 2 - 0.25*inch
            
            canvas.drawInlineImage(img, x, y, width=scaled_width, height=scaled_height)


class SParameterPlotter:
    """Generates S-parameter plots"""
    
    def __init__(self, network):
        """
        Parameters
        ----------
        network : skrf.Network
            The network to plot (original, not DC-extrapolated)
        """
        self.network = network
        
    def generate_figure(self, title="S-Parameters"):
        """
        Generate matplotlib figure for S-parameters
        
        Returns
        -------
        BytesIO
            Buffer containing PNG image data
        """
        dut = self.network
        gd = dut.group_delay
        
        fig = plt.figure(figsize=(11, 8.5))
        gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.4)
        
        # S11 - Top Left
        ax_s11 = fig.add_subplot(gs[0, 0])
        ax_s11_gd = ax_s11.twinx()
        dut.plot_s_db(m=0, n=0, ax=ax_s11, color='C0')
        ax_s11.set_ylabel('S11 (dB)', color='C0')
        ax_s11.tick_params(axis='y', labelcolor='C0')
        ax_s11_gd.plot(dut.f/1e9, gd[:, 0, 0]*1e9, color='C1')
        ax_s11_gd.set_ylabel('Group Delay (ns)', color='C1')
        ax_s11_gd.tick_params(axis='y', labelcolor='C1')
        ax_s11.set_title('S11')
        ax_s11.grid(True)
        
        # S22 - Top Right
        ax_s22 = fig.add_subplot(gs[0, 1])
        ax_s22_gd = ax_s22.twinx()
        dut.plot_s_db(m=1, n=1, ax=ax_s22, color='C0')
        ax_s22.set_ylabel('S22 (dB)', color='C0')
        ax_s22.tick_params(axis='y', labelcolor='C0')
        ax_s22_gd.plot(dut.f/1e9, gd[:, 1, 1]*1e9, color='C1')
        ax_s22_gd.set_ylabel('Group Delay (ns)', color='C1')
        ax_s22_gd.tick_params(axis='y', labelcolor='C1')
        ax_s22.set_title('S22')
        ax_s22.grid(True)
        
        # S21 - Bottom Half
        ax_s21 = fig.add_subplot(gs[1, :])
        ax_s21_gd = ax_s21.twinx()
        dut.plot_s_db(m=1, n=0, ax=ax_s21, color='C0')
        ax_s21.set_ylabel('S21 (dB)', color='C0')
        ax_s21.tick_params(axis='y', labelcolor='C0')
        ax_s21_gd.plot(dut.f/1e9, gd[:, 1, 0]*1e9, color='C1')
        ax_s21_gd.set_ylabel('Group Delay (ns)', color='C1')
        ax_s21_gd.tick_params(axis='y', labelcolor='C1')
        ax_s21.set_xlabel('Frequency (GHz)')
        ax_s21.set_title('S21')
        ax_s21.grid(True)
        
        plt.suptitle(title, fontsize=14, fontweight='bold')
        
        # Save to buffer
        buf = BytesIO()
        plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
        buf.seek(0)
        plt.close(fig)
        
        return buf


class TDRPlotter:
    """Generates TDR/TDT plots"""
    
    def __init__(self, network_dc):
        """
        Parameters
        ----------
        network_dc : skrf.Network
            The DC-extrapolated network for time domain analysis
        """
        self.network_dc = network_dc
        
    def generate_figure(self, title="TDR/TDT"):
        """
        Generate matplotlib figure for TDR/TDT (compact for table space)
        
        Returns
        -------
        BytesIO
            Buffer containing PNG image data
        """
        dut_dc = self.network_dc
        
        # Smaller figure to leave room for table
        fig = plt.figure(figsize=(11, 6))
        gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.4)
        
        # Z11 - Top Left
        ax_z11 = fig.add_subplot(gs[0, 0])
        dut_dc.plot_z_time_db(m=0, n=0, ax=ax_z11)
        ax_z11.set_title('Z11 - TDR')
        ax_z11.set_xlabel('Time (ns)')
        ax_z11.set_ylabel('Impedance (Ohms)')
        ax_z11.grid(True)
        
        # Z22 - Top Right (time reversed)
        ax_z22 = fig.add_subplot(gs[0, 1])
        dut_dc.plot_z_time_db(m=1, n=1, ax=ax_z22)
        ax_z22.invert_xaxis()
        ax_z22.set_title('Z22 - TDR (Time Reversed)')
        ax_z22.set_xlabel('Time (ns)')
        ax_z22.set_ylabel('Impedance (Ohms)')
        ax_z22.grid(True)
        
        # Z21 - Bottom Half (impulse and step)
        ax_z21 = fig.add_subplot(gs[1, :])
        dut_dc.plot_s_time_impulse(m=1, n=0, ax=ax_z21, label='Impulse Response', color='C0')
        dut_dc.plot_z_time_step(m=1, n=0, ax=ax_z21, label='Step Response', color='C1')
        ax_z21.set_title('Z21 - TDT')
        ax_z21.set_xlabel('Time (ns)')
        ax_z21.set_ylabel('Response')
        ax_z21.legend(loc='upper right')
        ax_z21.grid(True)
        
        plt.suptitle(title, fontsize=14, fontweight='bold')
        
        # Save to buffer
        buf = BytesIO()
        plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
        buf.seek(0)
        plt.close(fig)
        
        return buf
    
    def extract_metrics(self, target_impedance=50.0, tolerance=5.0):
        """
        Extract max impedance values and calculate margins
        
        Parameters
        ----------
        target_impedance : float
            Target impedance in Ohms (default 50)
        tolerance : float
            Acceptable tolerance in Ohms for pass/fail (default ±5)
            
        Returns
        -------
        pd.DataFrame
            DataFrame with metrics for Z11, Z22, Z21
        """
        dut_dc = self.network_dc
        
        # Get time-domain impedance data
        z_time = dut_dc.z_time  # Shape: (nfreq, nports, nports)
        
        # Find max impedance magnitudes
        z11_max = np.max(np.abs(z_time[:, 0, 0]))
        z22_max = np.max(np.abs(z_time[:, 1, 1]))
        z21_max = np.max(np.abs(z_time[:, 1, 0]))
        
        # Calculate margins from target
        z11_margin = z11_max - target_impedance
        z22_margin = z22_max - target_impedance
        z21_margin = z21_max - target_impedance
        
        # Determine pass/fail based on tolerance
        z11_pass = abs(z11_margin) <= tolerance
        z22_pass = abs(z22_margin) <= tolerance
        z21_pass = abs(z21_margin) <= tolerance
        
        # Build DataFrame with new column order
        data = {
            'Parameter': ['Z11', 'Z22', 'Z21'],
            'Target (Ω)': [f'{target_impedance:.2f}'] * 3,
            'Reading (Ω)': [f'{z11_max:.2f}', f'{z22_max:.2f}', f'{z21_max:.2f}'],
            'Margin (Ω)': [f'{z11_margin:+.2f}', f'{z22_margin:+.2f}', f'{z21_margin:+.2f}'],
            'Pass/Fail': ['PASS' if z11_pass else 'FAIL',
                         'PASS' if z22_pass else 'FAIL',
                         'PASS' if z21_pass else 'FAIL']
        }
        
        df = pd.DataFrame(data)
        return df


class SParameterPage:
    """Page layout for S-parameter plots"""
    
    def __init__(self, figure_buffer, dut_name):
        """
        Parameters
        ----------
        figure_buffer : BytesIO
            Pre-generated figure buffer from SParameterPlotter
        dut_name : str
            Name/identifier for this DUT
        """
        if figure_buffer is None:
            raise ValueError("figure_buffer cannot be None - generate plot first")
        
        self.figure_buffer = figure_buffer
        self.dut_name = dut_name
        
    def add_to_canvas(self, canvas, header_link=None):
        """
        Add this page to a ReportLab canvas
        
        Parameters
        ----------
        canvas : reportlab.pdfgen.canvas.Canvas
            The PDF canvas to draw on
        header_link : str, optional
            URL or bookmark name for header hyperlink
        """
        page_width, page_height = letter
        
        # Add header with optional hyperlink
        if header_link:
            canvas.linkURL(header_link, (0.5*inch, page_height - 0.4*inch, 
                                         page_width - 0.5*inch, page_height - 0.2*inch))
        
        canvas.setFont("Helvetica-Bold", 10)
        canvas.drawString(0.5*inch, page_height - 0.3*inch, 
                         f"S-Parameters: {self.dut_name}")
        
        # Load and draw image
        with Image.open(self.figure_buffer) as img:
            img_width, img_height = img.size
            
            margin = 0.5 * inch
            max_width = page_width - 2*margin
            max_height = page_height - 2*margin - 0.5*inch  # Extra space for header
            
            scale = min(max_width / img_width, max_height / img_height)
            scaled_width = img_width * scale
            scaled_height = img_height * scale
            
            x = (page_width - scaled_width) / 2
            y = (page_height - scaled_height) / 2 - 0.25*inch  # Shift down for header
            
            canvas.drawInlineImage(img, x, y, width=scaled_width, height=scaled_height)


class TDRPage:
    """Page layout for TDR/TDT plots with metrics table"""
    
    def __init__(self, figure_buffer, metrics_df, dut_name):
        """
        Parameters
        ----------
        figure_buffer : BytesIO
            Pre-generated figure buffer from TDRPlotter
        metrics_df : pd.DataFrame
            DataFrame with impedance metrics
        dut_name : str
            Name/identifier for this DUT
        """
        if figure_buffer is None:
            raise ValueError("figure_buffer cannot be None - generate plot first")
        if metrics_df is None:
            raise ValueError("metrics_df cannot be None")
        
        self.figure_buffer = figure_buffer
        self.metrics_df = metrics_df
        self.dut_name = dut_name
        
    def add_to_canvas(self, canvas, header_link=None, table_link=None):
        """
        Add this page to a ReportLab canvas
        
        Parameters
        ----------
        canvas : reportlab.pdfgen.canvas.Canvas
            The PDF canvas to draw on
        header_link : str, optional
            URL or bookmark name for header hyperlink
        table_link : str, optional
            URL or bookmark name for table title hyperlink
        """
        page_width, page_height = letter
        
        # Add header with optional hyperlink
        if header_link:
            canvas.linkURL(header_link, (0.5*inch, page_height - 0.4*inch, 
                                         page_width - 0.5*inch, page_height - 0.2*inch))
        
        canvas.setFont("Helvetica-Bold", 10)
        canvas.drawString(0.5*inch, page_height - 0.3*inch, 
                         f"TDR/TDT: {self.dut_name}")
        
        # Load and draw image (shifted up)
        with Image.open(self.figure_buffer) as img:
            img_width, img_height = img.size
            
            margin = 0.5 * inch
            # Reserve space for table at bottom
            table_space = 1.8 * inch
            max_width = page_width - 2*margin
            max_height = page_height - 2*margin - 0.5*inch - table_space
            
            scale = min(max_width / img_width, max_height / img_height)
            scaled_width = img_width * scale
            scaled_height = img_height * scale
            
            # Position image higher up
            x = (page_width - scaled_width) / 2
            y = page_height - scaled_height - 0.75*inch  # Near top
            
            canvas.drawInlineImage(img, x, y, width=scaled_width, height=scaled_height)
        
        # Add metrics table below plots
        self._add_metrics_table(canvas, y - 0.3*inch, table_link)
    
    def _add_metrics_table(self, canvas, y_position, table_link=None):
        """
        Add metrics table to canvas
        
        Parameters
        ----------
        canvas : reportlab.pdfgen.canvas.Canvas
            The PDF canvas
        y_position : float
            Y position for top of table
        table_link : str, optional
            URL or bookmark name for table title hyperlink
        """
        page_width, page_height = letter
        
        # Add table title with optional hyperlink
        canvas.setFont("Helvetica-Bold", 11)
        title_y = y_position
        
        if table_link:
            canvas.linkURL(table_link, (2*inch, title_y - 0.15*inch, 
                                        page_width - 2*inch, title_y + 0.15*inch))
        
        canvas.drawCentredString(page_width/2, title_y, "Impedance Metrics Summary")
        
        # Convert DataFrame to list of lists for ReportLab Table
        data = [self.metrics_df.columns.tolist()] + self.metrics_df.values.tolist()
        
        # Create table
        table = Table(data, colWidths=[1.2*inch, 1.2*inch, 1.4*inch, 1.2*inch, 1.2*inch])
        
        # Build base style
        style_commands = [
            # Header row
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#4A90E2')),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 11),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('TOPPADDING', (0, 0), (-1, 0), 12),
            
            # Data rows - default style
            ('BACKGROUND', (0, 1), (-1, -1), colors.white),
            ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 1), (-1, -1), 10),
            ('ALIGN', (0, 1), (-1, -1), 'CENTER'),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('TOPPADDING', (0, 1), (-1, -1), 8),
            ('BOTTOMPADDING', (0, 1), (-1, -1), 8),
        ]
        
        # Add red background for failed rows
        for i, row in enumerate(self.metrics_df.values, start=1):
            if row[-1] == 'FAIL':  # Last column is Pass/Fail
                style_commands.append(
                    ('BACKGROUND', (0, i), (-1, i), colors.HexColor('#FFB3B3'))
                )
        
        table.setStyle(TableStyle(style_commands))
        
        # Calculate table position (centered horizontally)
        table_width, table_height = table.wrap(0, 0)
        x_position = (page_width - table_width) / 2
        
        # Draw table below title
        table.drawOn(canvas, x_position, title_y - table_height - 0.3*inch)


def add_watermark(canvas, text="CONFIDENTIAL TEST DATA"):
    """
    Add diagonal watermark to canvas page
    
    Parameters
    ----------
    canvas : reportlab.pdfgen.canvas.Canvas
        The PDF canvas
    text : str
        Watermark text to display
    """
    page_width, page_height = letter
    
    # Save the current canvas state
    canvas.saveState()
    
    # Set watermark properties
    canvas.setFont("Helvetica-Bold", 60)
    canvas.setFillColorRGB(0.9, 0.9, 0.9, alpha=0.3)  # Light gray, semi-transparent
    
    # Calculate center position
    center_x = page_width / 2
    center_y = page_height / 2
    
    # Rotate and draw text
    canvas.translate(center_x, center_y)
    canvas.rotate(45)  # 45 degree diagonal
    
    # Center the text at the rotation point
    text_width = canvas.stringWidth(text, "Helvetica-Bold", 60)
    canvas.drawString(-text_width / 2, 0, text)
    
    # Restore canvas state
    canvas.restoreState()


def create_toc_page(canvas, dut_list):
    """
    Create a simple Table of Contents page
    
    Parameters
    ----------
    canvas : reportlab.pdfgen.canvas.Canvas
        The PDF canvas
    dut_list : list of str
        List of DUT names
    """
    page_width, page_height = letter
    
    # Title
    canvas.setFont("Helvetica-Bold", 20)
    canvas.drawCentredString(page_width/2, page_height - 2*inch, 
                            "VNA S-Parameter Report")
    
    # TOC Header
    canvas.setFont("Helvetica-Bold", 14)
    canvas.drawString(1.5*inch, page_height - 3*inch, "Table of Contents")
    
    # TOC Entries
    canvas.setFont("Helvetica", 11)
    y_position = page_height - 3.5*inch
    page_num = 2  # Start after TOC
    
    # VNA Configuration
    canvas.drawString(1.5*inch, y_position, "VNA Configuration")
    canvas.drawRightString(page_width - 1.5*inch, y_position, f"Page {page_num}")
    y_position -= 0.5*inch
    page_num += 1
    
    # Diagrams
    canvas.drawString(1.5*inch, y_position, "S-Parameter Diagrams")
    canvas.drawRightString(page_width - 1.5*inch, y_position, f"Pages {page_num}-{page_num+3}")
    y_position -= 0.5*inch
    page_num += 4
    
    # DUT measurements
    for dut_name in dut_list:
        # S-Parameter entry
        canvas.drawString(1.5*inch, y_position, 
                         f"{dut_name} - S-Parameters")
        canvas.drawRightString(page_width - 1.5*inch, y_position, 
                              f"Page {page_num}")
        y_position -= 0.3*inch
        page_num += 1
        
        # TDR/TDT entry
        canvas.drawString(1.5*inch, y_position, 
                         f"{dut_name} - TDR/TDT")
        canvas.drawRightString(page_width - 1.5*inch, y_position, 
                              f"Page {page_num}")
        y_position -= 0.5*inch
        page_num += 1


def main():
    """
    Main function to generate the complete VNA report
    """
    # ============================================
    # CREATE TEST NETWORK AND VNA METADATA
    # ============================================
    
    # Create test network
    freq = rf.Frequency(start=1, stop=10, npoints=101, unit='GHz')
    line = rf.media.DefinedGammaZ0(frequency=freq, z0=50)
    tline = line.line(d=1.0, unit='cm', name='test_line')
    
    dut = tline
    dut_dc = dut.extrapolate_to_dc(kind='linear')
    
    # Create VNA metadata
    vna_metadata = VNAMetadata(
        instrument_id="Keysight N5247B PNA-X",
        if_bandwidth=10e3,  # 10 kHz
        num_averages=16,
        num_points=101,
        start_freq=1e9,  # 1 GHz
        stop_freq=10e9,  # 10 GHz
        power_level=-10.0,  # -10 dBm
        calibration_type="SOLT",
        calibration_date="2026-01-20",
        operator="Test Engineer",
        notes="Test measurement for demonstration purposes. Cable length approximately 1 cm."
    )
    
    # Print repr to console for "shits and giggles"
    print("\n" + "="*70)
    print("VNA Metadata __repr__ output:")
    print("="*70)
    print(repr(vna_metadata))
    print("="*70 + "\n")
    
    # ============================================
    # CREATE PLOTTERS
    # ============================================
    
    sp_plotter = SParameterPlotter(dut)
    tdr_plotter = TDRPlotter(dut_dc)
    diagram_gen = DifferentialPairDiagram()
    
    # ============================================
    # GENERATE ALL CONTENT (FAIL FAST)
    # ============================================
    
    dut_names = [f"DUT_{i+1}" for i in range(5)]
    sp_figures = {}
    tdr_figures = {}
    tdr_metrics = {}
    
    print("Generating plots and extracting metrics...")
    for dut_name in dut_names:
        print(f"  {dut_name}...")
        sp_figures[dut_name] = sp_plotter.generate_figure(title=f"DUT: {dut_name} - S-Parameters")
        tdr_figures[dut_name] = tdr_plotter.generate_figure(title=f"DUT: {dut_name} - TDR/TDT")
        tdr_metrics[dut_name] = tdr_plotter.extract_metrics(target_impedance=50.0, tolerance=5.0)
    
    print("Generating diagrams...")
    twoport_diag = diagram_gen.generate_twoport_diagram()
    single_ended_diag = diagram_gen.generate_single_ended_diagram()
    crosstalk_diag = diagram_gen.generate_crosstalk_diagram()
    mixed_mode_diag = diagram_gen.generate_mixed_mode_diagram()
    
    # ============================================
    # BUILD PDF REPORT - TABLE FORMAT
    # ============================================
    
    print("Building PDF with table format...")
    
    output_filename_table = "vna_report_table_format.pdf"
    c = pdf_canvas.Canvas(output_filename_table, pagesize=letter)
    
    try:
        # Page 1: TOC
        create_toc_page(c, dut_names)
        add_watermark(c)
        c.showPage()
        
        # Page 2: VNA Configuration (TABLE FORMAT)
        vna_page = VNAMetadataPage(vna_metadata, use_repr_format=False)
        vna_page.add_to_canvas(c, header_link="#TOC")
        add_watermark(c)
        c.showPage()
        
        # Pages 3-6: Diagrams
        diag_page0 = DiagramPage(twoport_diag, "S-Parameter Configuration: 2-Port Device")
        diag_page0.add_to_canvas(c, header_link="#TOC")
        add_watermark(c)
        c.showPage()
        
        diag_page1 = DiagramPage(single_ended_diag, "S-Parameter Configuration: Single-Ended")
        diag_page1.add_to_canvas(c, header_link="#TOC")
        add_watermark(c)
        c.showPage()
        
        diag_page2 = DiagramPage(crosstalk_diag, "S-Parameter Configuration: Cross-Coupling")
        diag_page2.add_to_canvas(c, header_link="#TOC")
        add_watermark(c)
        c.showPage()
        
        diag_page3 = DiagramPage(mixed_mode_diag, "S-Parameter Configuration: Mixed-Mode")
        diag_page3.add_to_canvas(c, header_link="#TOC")
        add_watermark(c)
        c.showPage()
        
        # Pages 7+: DUT pages (2 pages per DUT)
        for dut_name in dut_names:
            # S-Parameter page
            sp_page = SParameterPage(sp_figures[dut_name], dut_name)
            sp_page.add_to_canvas(c, header_link="#TOC")
            add_watermark(c)
            c.showPage()
            
            # TDR/TDT page with metrics table
            tdr_page = TDRPage(tdr_figures[dut_name], tdr_metrics[dut_name], dut_name)
            tdr_page.add_to_canvas(c, header_link="#TOC", table_link="#metrics")
            add_watermark(c)
            c.showPage()
        
        # Save PDF
        c.save()
        
        total_pages = 1 + 1 + 4 + len(dut_names) * 2
        print(f"\n✓ Report saved: {output_filename_table}")
        print(f"✓ Format: Table layout")
        print(f"✓ Total pages: {total_pages}")
    
    except Exception as e:
        print(f"✗ Error building table format report: {e}")
        raise
    
    # ============================================
    # BUILD PDF REPORT - REPR FORMAT
    # ============================================
    
    print("\nBuilding PDF with repr/dotted format...")
    
    output_filename_repr = "vna_report_repr_format.pdf"
    c = pdf_canvas.Canvas(output_filename_repr, pagesize=letter)
    
    try:
        # Page 1: TOC
        create_toc_page(c, dut_names)
        add_watermark(c)
        c.showPage()
        
        # Page 2: VNA Configuration (REPR FORMAT - for shits and giggles!)
        vna_page = VNAMetadataPage(vna_metadata, use_repr_format=True)
        vna_page.add_to_canvas(c, header_link="#TOC")
        add_watermark(c)
        c.showPage()
        
        # Pages 3-6: Diagrams
        diag_page0 = DiagramPage(twoport_diag, "S-Parameter Configuration: 2-Port Device")
        diag_page0.add_to_canvas(c, header_link="#TOC")
        add_watermark(c)
        c.showPage()
        
        diag_page1 = DiagramPage(single_ended_diag, "S-Parameter Configuration: Single-Ended")
        diag_page1.add_to_canvas(c, header_link="#TOC")
        add_watermark(c)
        c.showPage()
        
        diag_page2 = DiagramPage(crosstalk_diag, "S-Parameter Configuration: Cross-Coupling")
        diag_page2.add_to_canvas(c, header_link="#TOC")
        add_watermark(c)
        c.showPage()
        
        diag_page3 = DiagramPage(mixed_mode_diag, "S-Parameter Configuration: Mixed-Mode")
        diag_page3.add_to_canvas(c, header_link="#TOC")
        add_watermark(c)
        c.showPage()
        
        # Pages 7+: DUT pages (2 pages per DUT)
        for dut_name in dut_names:
            # S-Parameter page
            sp_page = SParameterPage(sp_figures[dut_name], dut_name)
            sp_page.add_to_canvas(c, header_link="#TOC")
            add_watermark(c)
            c.showPage()
            
            # TDR/TDT page with metrics table
            tdr_page = TDRPage(tdr_figures[dut_name], tdr_metrics[dut_name], dut_name)
            tdr_page.add_to_canvas(c, header_link="#TOC", table_link="#metrics")
            add_watermark(c)
            c.showPage()
        
        # Save PDF
        c.save()
        
        print(f"\n✓ Report saved: {output_filename_repr}")
        print(f"✓ Format: Repr/dotted-leader layout")
        print(f"✓ Total pages: {total_pages}")
        print(f"\n{'='*70}")
        print("Summary:")
        print(f"  - {output_filename_table} (professional table format)")
        print(f"  - {output_filename_repr} (dotted-leader format, for shits and giggles)")
        print(f"{'='*70}\n")
    
    except Exception as e:
        print(f"✗ Error building repr format report: {e}")
        raise


if __name__ == "__main__":
    main()


VNA Metadata __repr__ output:
VNA MEASUREMENT CONFIGURATION

Instrument.................... Keysight N5247B PNA-X
Start Frequency............... 1.000 GHz
Stop Frequency................ 10.000 GHz
Frequency Span................ 9.000 GHz
Frequency Step................ 90.000 MHz
Number of Points.............. 101
IF Bandwidth.................. 10.0 kHz
Number of Averages............ 16
Power Level................... -10.0 dBm
Calibration Type.............. SOLT
Calibration Date.............. 2026-01-20
Measurement Date.............. 2026-01-26 20:00:44
Operator...................... Test Engineer

Notes:
------------------------------------------------------------
Test measurement for demonstration purposes. Cable length approximately 1 cm.

Generating plots and extracting metrics...
  DUT_1...


/home/ukde/.pyenv/versions/SB/lib/python3.13/site-packages/skrf/mathFunctions.py:268: RuntimeWarning: divide by zero encountered in log10
  out = 20 * np.log10(z)
/home/ukde/.pyenv/versions/SB/lib/python3.13/site-packages/matplotlib/cbook.py:1719: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/home/ukde/.pyenv/versions/SB/lib/python3.13/site-packages/matplotlib/cbook.py:1355: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


  DUT_2...
  DUT_3...
  DUT_4...
  DUT_5...
Generating diagrams...
Building PDF with table format...

✓ Report saved: vna_report_table_format.pdf
✓ Format: Table layout
✓ Total pages: 16

Building PDF with repr/dotted format...

✓ Report saved: vna_report_repr_format.pdf
✓ Format: Repr/dotted-leader layout
✓ Total pages: 16

Summary:
  - vna_report_table_format.pdf (professional table format)
  - vna_report_repr_format.pdf (dotted-leader format, for shits and giggles)

